In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from PIL import Image
import re

In [2]:
def preprocess_data(csv_file):
    data_dict = {}
    df = pd.read_csv(csv_file)
    try:
        header = df.columns[0]
        match = re.search(r'(\d+)dB', header.split('_')[1])
    except Exception as e:
        print(f"Error processing header '{header}': {header}. Skipping this column.")
    # Check if there is a match for the numeric part in the header
    if match:
        if header.split('_')[1].startswith('N') or header.split('_')[1].startswith('n'):
            SNR_level = -int(match.group(1))
        else:
            # If numeric part found, extract the SNR level
            SNR_level = int(match.group(1))
    else:
        # If no numeric part and no 'N', set SNR level to 0
        SNR_level = 0
    modulation_type = header.split('_')[0]
    fading = int(re.search(r'(\d+)Hz', header.split('_')[2]).group(1))
    key = tuple((modulation_type, SNR_level, fading))
    # Extract the column data from the DataFrame
    complex_column = df[header]
    try:
        complex_column = complex_column.apply(lambda x: re.sub(r'i.*$', 'i', x))
        complex_column = complex_column.apply(lambda x: x.replace('i', 'j') if ('i' in x or 'j' in x) and '.' in x else 0)
        complex_column = complex_column.apply(lambda x: complex(x))
    except ValueError as e:
        print(f"Error processing header '{header}': {e}.")
    real_part = complex_column.apply(lambda x: x.real).astype(np.float32)
    imaginary_part = complex_column.apply(lambda x: x.imag).astype(np.float32)
    
    data_dict[key] = np.array([real_part, imaginary_part])

    print(data_dict[key].shape)
    for key, value in data_dict.items():
            
        # Transpose signal data to shape (*, 128, 2)
        data_dict[key] = np.reshape(value.T, (1, 128, 2))
    print(data_dict[key].shape)

    return data_dict

In [3]:
data_dict = preprocess_data('BPSK_15dB_5Hz.csv')

(2, 128)
(1, 128, 2)


In [4]:
model = load_model('cldnn_modelv11.keras')

In [5]:
data_dict.keys()

dict_keys([('BPSK', 15, 5)])

In [6]:
next(iter(data_dict))

('BPSK', 15, 5)

In [7]:
X = data_dict[next(iter(data_dict))]

In [8]:
predictions = np.argmax(model.predict(X), axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step


In [9]:
modulation_output_map = {0: 'BPSK',
                         1: 'FSK2',
                         2: 'FSK4',
                         3: 'FSK8',
                         4: 'LFM100K',
                         5: 'LFM10M',
                         6: 'LFM1M',
                         7: 'PSK8',
                         8: 'QAM16',
                         9: 'QAM256',
                         10: 'QAM8',
                         11: 'QPSK'}

In [10]:
modulation_predicted = modulation_output_map[predictions[0]]
modulation_predicted

'BPSK'

In [11]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# Example signal data with shape (1, 128, 2)
signal_data = np.random.randn(1, 128, 2)  # Replace this with your actual signal data

# Extract real and imaginary parts from the signal data
real_part = signal_data[0, :, 0]
imaginary_part = signal_data[0, :, 1]

# Create a new figure for plotting
plt.figure(figsize=(12, 6))

# Plot the real part as a blue line
plt.plot(np.arange(128), real_part, label='Real Part', color='g')

# Plot the imaginary part as a red dashed line
plt.plot(np.arange(128), imaginary_part, label='Imaginary Part', color='b', linestyle='--')

# Add labels, title, legend, and grid
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title(f'{}')
plt.grid(True)
plt.legend()

# Show the plot
plt.show()


SyntaxError: f-string: empty expression not allowed (511546419.py, line 23)

In [15]:
import numpy as np
import plotly.graph_objects as go

def plot_real_imaginary_plotly(signal_data, modulation_type):
    """
    Plot the real and imaginary parts of a signal data array using Plotly.

    Parameters:
    signal_data (ndarray): Input signal data array with shape (1, N, 2),
                           where N is the number of data points.

    Returns:
    None (displays the interactive plot)
    """
    # Extract real and imaginary parts from the signal data
    real_part = signal_data[0, :, 0]
    imaginary_part = signal_data[0, :, 1]

    # Create a Plotly figure for an interactive line plot
    fig = go.Figure()

    # Add a line trace for the real part
    fig.add_trace(go.Scatter(x=np.arange(len(real_part)), y=real_part,
                             mode='lines', name='Real Part', line=dict(color='blue')))

    # Add a line trace for the imaginary part
    fig.add_trace(go.Scatter(x=np.arange(len(imaginary_part)), y=imaginary_part,
                             mode='lines', name='Imaginary Part', line=dict(color='orange', dash='dash')))

    # Update layout properties
    fig.update_layout(title=modulation_type,
                      xaxis_title='Time',
                      yaxis_title='Amplitude',
                      hovermode='closest')

    # Show the interactive plot
    fig.show()

# Example usage:
# Generate example signal data (replace with your actual signal data)
signal_data = np.random.randn(1, 128, 2)
plot_real_imaginary_plotly(signal_data, 'BPSK')


In [16]:
pip install --upgrade nbformat

Note: you may need to restart the kernel to use updated packages.


In [21]:
A = next(iter(data_dict))
modulation_type = f'{A[0]}_{A[1]}dB_{A[2]}Hz'

In [22]:
modulation_type

'BPSK_15dB_5Hz'